In [1]:
import utils
import network
import argparse
import numpy as np
from alphabet import Alphabet

from datetime import datetime
import pickle5 as pickle

In [2]:
num_lstm_layer = int(2)
num_hidden_node = int(64)
dropout = float(0.5)
batch_size = int(50)
patience = int(3)
vector_dir = 'embedding/vectors.npy'
word_dir = 'embedding/words.pl'
train_dir = 'data/train_sample.txt'
dev_dir = 'data/dev_sample.txt '
test_dir = 'data/test_sample.txt'

In [16]:
datetime.now()

print('Loading data...')
# input_train, output_train, input_dev, output_dev, input_test, output_test, alphabet_tag, max_length = \
#     utils.create_data(word_dir, vector_dir, train_dir, dev_dir, test_dir)

Loading data...


In [17]:
def create_data(word_dir, vector_dir, train_dir, dev_dir, test_dir):
    embedd_vectors = np.load(vector_dir)
    with open(word_dir, 'rb') as handle:
        embedd_words = pickle.load(handle)
    embedd_dim = np.shape(embedd_vectors)[1]
    unknown_embedd = np.random.uniform(-0.01, 0.01, [1, embedd_dim])
    word_list_train, pos_list_train, chunk_list_train, tag_list_train, num_sent_train, max_length_train = \
        read_conll_format(train_dir)
    word_list_dev, pos_list_dev, chunk_list_dev, tag_list_dev, num_sent_dev, max_length_dev = \
        read_conll_format(dev_dir)
    word_list_test, pos_list_test, chunk_list_test, tag_list_test, num_sent_test, max_length_test = \
        read_conll_format(test_dir)
    pos_id_list_train, pos_id_list_dev, pos_id_list_test, chunk_id_list_train, chunk_id_list_dev, chunk_id_list_test, \
    tag_id_list_train, tag_id_list_dev, tag_id_list_test, alphabet_pos, alphabet_chunk, alphabet_tag = \
        map_string_2_id(pos_list_train, pos_list_dev, pos_list_test, chunk_list_train, chunk_list_dev, chunk_list_test,
                        tag_list_train, tag_list_dev, tag_list_test)
    max_length = max(max_length_train, max_length_dev, max_length_test)
    input_train, output_train, input_dev, output_dev, input_test, output_test = \
        create_vector_data(word_list_train, word_list_dev, word_list_test, pos_id_list_train, pos_id_list_dev,
                           pos_id_list_test, chunk_id_list_train, chunk_id_list_dev, chunk_id_list_test,
                           tag_id_list_train, tag_id_list_dev, tag_id_list_test, unknown_embedd, embedd_words,
                           embedd_vectors, embedd_dim, max_length, alphabet_pos.size(), alphabet_chunk.size(),
                           alphabet_tag.size())
    return input_train, output_train, input_dev, output_dev, input_test, tag_id_list_test, alphabet_tag, max_length

In [18]:
embedd_vectors = np.load(vector_dir)
with open(word_dir, 'rb') as handle:
    embedd_words = pickle.load(handle)
print(f"Embedd_vectors shape: {embedd_vectors.shape}")
print(f"Embedd_vectors instance: {embedd_vectors[0]}")
print(f"Embedd_words len: {len(embedd_words)}")
print(f"Embedd_words instance: {embedd_words[:4]}")

Embedd_vectors shape: (439670, 300)
Embedd_vectors instance: [-1.662e-03 -1.130e-04 -4.430e-04 -4.790e-04 -4.270e-04  8.960e-04
  1.370e-04  9.770e-04 -4.400e-04  1.440e-04 -8.830e-04 -5.030e-04
 -5.840e-04 -1.260e-04  1.196e-03  4.560e-04 -1.540e-04 -2.800e-05
  6.290e-04 -1.045e-03  1.087e-03  9.350e-04  3.620e-04  9.730e-04
  9.510e-04 -1.346e-03 -1.088e-03 -5.600e-05 -1.004e-03 -1.361e-03
  5.020e-04  5.260e-04 -1.520e-04 -1.052e-03 -1.015e-03 -3.300e-04
  3.540e-04 -5.850e-04  1.068e-03  8.900e-05  2.570e-04 -1.930e-04
 -6.400e-04 -4.290e-04  1.080e-03  1.150e-03  7.650e-04  1.229e-03
 -4.210e-04 -4.820e-04  7.470e-04  1.476e-03  4.410e-04 -1.240e-04
 -1.030e-03  8.070e-04  1.257e-03 -7.460e-04  1.900e-04  8.480e-04
  1.122e-03  1.930e-04 -7.640e-04 -5.740e-04 -7.870e-04  3.820e-04
  6.070e-04  4.410e-04  1.373e-03 -5.450e-04  1.543e-03  3.020e-04
  1.043e-03 -1.151e-03 -9.180e-04  1.043e-03 -7.580e-04 -9.710e-04
 -3.200e-05 -1.456e-03 -5.100e-05 -1.300e-05  5.830e-04 -6.960e-04
 

In [19]:
# read_conll_format()
from utils import *

word_list_train, pos_list_train, chunk_list_train, tag_list_train, num_sent_train, max_length_train = \
read_conll_format(train_dir)
word_list_dev, pos_list_dev, chunk_list_dev, tag_list_dev, num_sent_dev, max_length_dev = \
read_conll_format(dev_dir)
word_list_test, pos_list_test, chunk_list_test, tag_list_test, num_sent_test, max_length_test = \
read_conll_format(test_dir)

In [20]:
print(f"1st sentence in train set: {word_list_train[0]}")
print("-"*100)
print(f"1st poss in train set: {pos_list_train[0]}")
print("-"*100)
print(f"1st chunks in train set: {chunk_list_train[0]}")
print("-"*100)
print(f"1st tags in train set: {tag_list_train[0]}")
print("-"*100)
print(f"# sentence in train set:: {num_sent_train}")
print("-"*100)
print(f"max length of sent in train set: {max_length_train}")

1st sentence in train set: ['đó', 'là', 'con', 'đường', 'biển', 'ngắn', 'nhất', 'để', 'đi', 'từ', 'ấn_độ_dương', 'sang', 'thái_bình_dương', '<punct>', 'chiếm', 'đến', 'lượng', 'hàng_hoá', 'lưu_thông', 'đường_biển', 'của', 'thế_giới', '<punct>', 'đó', 'là', 'hải_trình', 'lớn', 'nhất', 'từ', 'tây', 'sang', 'đông', 'với', '<number>', 'lượt', 'tàu_bè', 'qua_lại', 'mỗi', 'năm', '<punct>']
----------------------------------------------------------------------------------------------------
1st poss in train set: ['P', 'V', 'Nc', 'N', 'N', 'A', 'A', 'E', 'V', 'E', 'NNP', 'V', 'NNP', 'CH', 'V', 'E', 'N', 'N', 'V', 'N', 'E', 'N', 'CH', 'P', 'V', 'N', 'A', 'R', 'E', 'N', 'V', 'N', 'E', 'M', 'Nc', 'N', 'V', 'L', 'N', 'CH']
----------------------------------------------------------------------------------------------------
1st chunks in train set: ['B-NP', 'B-VP', 'B-NP', 'B-NP', 'B-NP', 'B-AP', 'B-AP', 'B-PP', 'B-VP', 'B-PP', 'B-NP', 'B-VP', 'B-NP', 'O', 'B-VP', 'B-PP', 'B-NP', 'B-NP', 'B-VP', 'B-

In [21]:
pos_id_list_train, pos_id_list_dev, pos_id_list_test, chunk_id_list_train, chunk_id_list_dev, chunk_id_list_test, \
    tag_id_list_train, tag_id_list_dev, tag_id_list_test, alphabet_pos, alphabet_chunk, alphabet_tag = \
        map_string_2_id(pos_list_train, pos_list_dev, pos_list_test, chunk_list_train, chunk_list_dev, chunk_list_test,
                        tag_list_train, tag_list_dev, tag_list_test)

In [22]:
# map_string_2_id
print(f"# pos in train set: {alphabet_pos.next_index}")
print(f"pos instance: {alphabet_pos.get_instance(2)}")
print("-"*100)
print(f"# chunk in train set: {alphabet_chunk.next_index}")
print(f"chunk instance: {alphabet_chunk.get_instance(2)}")
print("-"*100)
print(f"# tag in train set: {alphabet_tag.next_index}")
print(f"tag instance: {alphabet_tag.get_instance(1)}")

# pos in train set: 19
pos instance: V
----------------------------------------------------------------------------------------------------
# chunk in train set: 8
chunk instance: B-VP
----------------------------------------------------------------------------------------------------
# tag in train set: 8
tag instance: O


In [27]:
max_length = max(max_length_train, max_length_dev, max_length_test)

In [28]:
print(f"max length: {max_length}")

max length: 43


In [29]:
embedd_dim = np.shape(embedd_vectors)[1]
unknown_embedd = np.random.uniform(-0.01, 0.01, [1, embedd_dim])
input_train, output_train, input_dev, output_dev, input_test, output_test = \
        create_vector_data(word_list_train, word_list_dev, word_list_test, pos_id_list_train, pos_id_list_dev,
                           pos_id_list_test, chunk_id_list_train, chunk_id_list_dev, chunk_id_list_test,
                           tag_id_list_train, tag_id_list_dev, tag_id_list_test, unknown_embedd, embedd_words,
                           embedd_vectors, embedd_dim, max_length, alphabet_pos.size(), alphabet_chunk.size(),
                           alphabet_tag.size())

In [30]:
# construct_tensor_word
word_train = construct_tensor_word(word_list_train, unknown_embedd, embedd_words, embedd_vectors, embedd_dim, 
                                       max_length)
word_dev = construct_tensor_word(word_list_dev, unknown_embedd, embedd_words, embedd_vectors, embedd_dim,
                                     max_length)
word_test = construct_tensor_word(word_list_test, unknown_embedd, embedd_words, embedd_vectors, embedd_dim,
                                      max_length)

In [31]:
print(f"input shape: {word_train.shape}")

input shape: (10, 43, 300)


In [34]:
# construct_tensor_onehot
dim_pos, dim_chunk, dim_tag = alphabet_pos.size(), alphabet_chunk.size(),alphabet_tag.size()
pos_train = construct_tensor_onehot(pos_id_list_train, max_length, dim_pos)
pos_dev = construct_tensor_onehot(pos_id_list_dev, max_length, dim_pos)
pos_test = construct_tensor_onehot(pos_id_list_test, max_length, dim_pos)
chunk_train = construct_tensor_onehot(chunk_id_list_train, max_length, dim_chunk)
chunk_dev = construct_tensor_onehot(chunk_id_list_dev, max_length, dim_chunk)
chunk_test = construct_tensor_onehot(chunk_id_list_test, max_length, dim_chunk)
tag_train = construct_tensor_onehot(tag_id_list_train, max_length, dim_tag)
tag_dev = construct_tensor_onehot(tag_id_list_dev, max_length, dim_tag)
tag_test = construct_tensor_onehot(tag_id_list_test, max_length, dim_tag)

In [35]:
print(f"pos_train shape: {pos_train.shape}")
print(f"pos instance: {pos_train[0,:,:]}")

pos_train shape: (10, 43, 19)
pos instance: [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

In [74]:
input_train = word_train
input_train = np.concatenate((input_train, pos_train), axis=2)
input_train = np.concatenate((input_train, chunk_train), axis=2)
output_train = tag_train
input_dev = word_dev
input_dev = np.concatenate((input_dev, pos_dev), axis=2)
input_dev = np.concatenate((input_dev, chunk_dev), axis=2)
output_dev = tag_dev
input_test = word_test
input_test = np.concatenate((input_test, pos_test), axis=2)
input_test = np.concatenate((input_test, chunk_test), axis=2)
output_test = tag_test

In [76]:
print(f"input_train shape: {input_train.shape}")

input_train shape: (10, 43, 327)


In [3]:
import infer

In [4]:
vector_dir = 'embedding/vectors.npy'
word_dir = 'embedding/words.pl'
test_dir = 'test.txt'
infer.infer_to_file(word_dir, vector_dir, test_dir, 'output3.txt')

C:\Users\thuyt\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [9]:
from underthesea import chunk

text = 'Bác_sĩ bây_giờ có_thể thản_nhiên báo tin bệnh_nhân bị ung_thư?'
ckn = chunk(text)

In [10]:
ckn

[('Bác_sĩ', 'Np', 'B-NP'),
 ('bây_giờ', 'V', 'B-VP'),
 ('có_thể', 'N', 'B-NP'),
 ('thản_nhiên', 'N', 'B-NP'),
 ('báo', 'V', 'B-VP'),
 ('tin', 'N', 'B-NP'),
 ('bệnh_nhân', 'N', 'B-NP'),
 ('bị', 'V', 'B-VP'),
 ('ung_thư', 'V', 'B-VP'),
 ('?', 'CH', 'O')]

In [ ]:
LINK = r'../../../NER/phonlp_data/train.txt'

def read_sent(input_file):
    with codecs.open(input_file, 'r', 'utf-8') as f:
        sent_list = []
        chunk_list = []
        pos_list = []
        tag_list = []
        words = []
        chunks = []
        poss = []
        tags = []
        num_sent = 0
        max_length = 0
        for line in f:
            line = line.split()
            if len(line) > 0:
                words.append(map_number_and_punct(line[0].lower()))
                poss.append(line[1])
                chunks.append(line[2])
                tags.append(line[3])
            else:
                word_list.append(words)
                pos_list.append(poss)
                chunk_list.append(chunks)
                tag_list.append(tags)
                sent_length = len(words)
                words = []
                chunks = []
                poss = []
                tags = []
                num_sent += 1
                max_length = max(max_length, sent_length)
    return word_list, pos_list, chunk_list, tag_list, num_sent, max_length